In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install webdriver-manager
!pip install requests beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 23.4 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64

In [ ]:
import time
import re
import csv
import logging
import os
import queue
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse, urljoin
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd
import html

class UOBScraper:
    def __init__(self, headless=True, output_dir="uob_data", max_workers=5):
        self.output_dir = output_dir
        self.max_workers = max_workers  # Number of parallel workers
        os.makedirs(output_dir, exist_ok=True)

        self.options = Options()
        if headless:
            self.options.add_argument("--headless")
        self.options.add_argument("--disable-gpu")
        self.options.add_argument("--no-sandbox")
        self.options.add_argument("--disable-dev-shm-usage")
        self.options.add_argument("--window-size=1920,1080")

        # Create a lock for thread-safe operations
        self.data_lock = threading.Lock()

        # Initialize empty data list
        self.data = []

        # Cache of visited URLs to avoid duplicated work
        self.visited_urls = set()

        # Setup cleaning patterns
        self._setup_cleaning_patterns()

    def _setup_cleaning_patterns(self):
        # Expanded list of boilerplate patterns
        self.boilerplate_patterns = [
            r"Terms and Conditions Apply",
            r"Click here.*",
            r"Learn More",
            r"Read more.*",
            r"Like us on Facebook",
            r"Follow us on Twitter",
            r"Back to Top",
            r"\*Terms and conditions apply.*",
            r"Back to Main.*",
            r"HOME",
            r"UOB Singapore",
            r"Current Promotions",
            r"Customer Service",
            r"Share Your Experiences",
            r"Copyright © \d{4} United Overseas Bank Limited",
            r"Co\. Reg\. No\. \d+Z\. All Rights Reserved\.",
            r"We use cookies in order to provide you with better services.*",
            r"By continuing to browse the site, you agree to our privacy notice and cookie policy\.",
            r"privacy notice",
            r"cookie policy",
            r"Back to MainHOME",
            r"Back to MainSave",
            r"Back to MainCards",
            r"Back to MainBorrow",
            r"Back to MainInvest",
            r"Back to MainInsure",
            r"Back to MainDigital Banking",
        ]

        # Encoding fix map
        self.encoding_fix_map = {
            "â€™": "'",   # Right single quotation mark
            "â€œ": '"',   # Left double quotation mark
            "â€": '"',    # Right double quotation mark
            "â€": "–",   # En dash
            "â€¢": "•",   # Bullet
            "Ã©": "é",    # Latin small letter e with acute
            "Ã¨": "è",    # Latin small letter e with grave
            "Ã ": "à",    # Latin small letter a with grave
            "Ã¢": "â",    # Latin small letter a with circumflex
            "Ã¤": "ä",    # Latin small letter a with diaeresis
            "Ã«": "ë",    # Latin small letter e with diaeresis
            "Ã¯": "ï",    # Latin small letter i with diaeresis
            "Ã´": "ô",    # Latin small letter o with circumflex
            "Ã¶": "ö",    # Latin small letter o with diaeresis
            "Ã¹": "ù",    # Latin small letter u with grave
            "Ã»": "û",    # Latin small letter u with circumflex
            "Ã¼": "ü",    # Latin small letter u with diaeresis
            "Ã§": "ç",    # Latin small letter c with cedilla
            "Â£": "£",    # Pound sign
            "Â€": "€",    # Euro sign
            "Â©": "©",    # Copyright sign
            "Â®": "®",    # Registered sign
            "â„¢": "™"    # Trademark sign
        }

    def clean_text(self, text):
        if not text:
            return ""

        # First decode HTML entities
        text = html.unescape(text)

        # Replace unwanted whitespace and non-breaking spaces
        text = re.sub(r'\s+', ' ', text).strip()
        text = text.replace('\xa0', ' ')

        # Fix encoding issues based on our mapping
        for wrong, correct in self.encoding_fix_map.items():
            text = text.replace(wrong, correct)

        # Apply additional Unicode normalization
        import unicodedata
        text = unicodedata.normalize('NFC', text)

        # Remove boilerplate texts using more aggressive pattern matching
        for pattern in self.boilerplate_patterns:
            text = re.sub(pattern, '', text, flags=re.IGNORECASE)

        # Remove menu navigation text patterns
        text = re.sub(r'Back to \w+', '', text, flags=re.IGNORECASE)

        # Final whitespace cleanup
        text = re.sub(r'\s+', ' ', text).strip()

        return text

    def is_valid_text(self, text):
        # More aggressive filtering of unwanted content
        if not text or len(text.strip()) <= 10:
            return False

        # Skip common footer/header/navigation text patterns
        unwanted_phrases = [
            "back to main", "copyright", "all rights reserved",
            "terms and conditions", "privacy policy", "cookie policy",
            "customer service", "promotions", "share your experiences"
        ]

        lower_text = text.lower()
        for phrase in unwanted_phrases:
            if phrase in lower_text:
                return False

        return True

    def extract_metadata_tags(self, url):
        tags = []
        if "insure" in url: tags.append("insurance")
        if "loan" in url or "borrow" in url: tags.append("loans")
        if "card" in url: tags.append("credit cards")
        if "save" in url: tags.append("savings")
        if "invest" in url: tags.append("investments")
        if "digital-banking" in url: tags.append("digital banking")
        if "wealth" in url: tags.append("wealth management")
        if "banking" in url: tags.append("banking")
        return ", ".join(tags) if tags else "general"

    def is_valid_url(self, url, base_url=None):
        """Check if URL should be processed based on content relevance."""
        if not url or url.startswith("javascript:"):
            return False

        # Convert relative URLs to absolute
        if base_url and url.startswith("/"):
            url = urljoin(base_url, url)

        # Filter by domain
        if "uob.com.sg" not in url:
            return False

        # Skip URLs that are already processed
        if url in self.visited_urls:
            return False

        # Skip URLs that are not related to banking products
        if "personal" not in url:
            return False

        # Skip file downloads
        if any(url.endswith(ext) for ext in ['.pdf', '.doc', '.docx', '.xls', '.xlsx', '.zip']):
            return False

        # Skip irrelevant sections
        skip_sections = ['careers', 'about', 'contact', 'newsroom', 'media', 'sustainability']
        if any(section in url for section in skip_sections):
            return False

        return True

    def scrape_page_with_driver(self, driver, url):
        """Scrape a single page using the provided driver."""
        try:
            # Mark URL as visited to avoid reprocessing
            self.visited_urls.add(url)

            # Try to navigate to the URL with a short timeout
            driver.get(url)

            try:
                # More efficient wait strategy: wait for specific elements that indicate page load
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )
            except TimeoutException:
                logging.warning(f"Timeout waiting for page to load: {url}")
                return [], []  # FIXED: Return two empty lists instead of one

            # No need for fixed sleep - wait for elements instead

            # Set page encoding to UTF-8
            driver.execute_script("document.charset='utf-8';")

            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Extract title from h1 or page title
            title_element = soup.find("h1")
            title = self.clean_text(title_element.get_text()) if title_element else self.clean_text(driver.title)

            # Attempt to extract a subtitle (h2 tag following the h1)
            subtitle = ""
            if title_element and title_element.find_next("h2"):
                subtitle = self.clean_text(title_element.find_next("h2").get_text())

            # Extract metadata tags from URL
            tag = self.extract_metadata_tags(url)

            # Collect page data
            page_data = []

            # Extract content from paragraphs and list items
            for section in soup.find_all(['p', 'li']):
                # Skip elements within navigation or footer sections
                if any(parent_tag in [parent.name for parent in section.parents] for parent_tag in ['nav', 'footer']):
                    continue

                # Skip elements with certain classes or IDs
                skip_classes = ['footer', 'navigation', 'menu', 'copyright', 'legal']
                if any(cls in section.get('class', []) for cls in skip_classes):
                    continue

                text = self.clean_text(section.get_text())
                if self.is_valid_text(text):
                    page_data.append({
                        "url": url,
                        "title": title,
                        "subtitle": subtitle,
                        "subheader": title,
                        "text": text,
                        "tag": tag
                    })

            # Find links for further crawling
            links = []
            for a in soup.find_all("a", href=True):
                try:
                    link = a["href"]
                    # Convert relative URLs to absolute
                    if link.startswith("/"):
                        link = urljoin(url, link)
                    # Only collect valid UOB URLs
                    if self.is_valid_url(link, url):
                        links.append(link)
                except (KeyError, AttributeError):
                    continue

            return page_data, links

        except Exception as e:
            logging.error(f"Error scraping {url}: {e}")
            return [], []  # Return two empty lists if there's an exception

    def scrape_page(self, url):
        """Scrape a single page with a new driver instance."""
        driver = webdriver.Chrome(options=self.options)
        try:
            page_data, _ = self.scrape_page_with_driver(driver, url)

            # Thread-safe addition of data
            with self.data_lock:
                self.data.extend(page_data)

            return page_data
        finally:
            driver.quit()

    def scrape_urls(self, urls):
        """Scrape a provided list of URLs in parallel."""
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # Submit all URL scraping tasks
            futures = [executor.submit(self.scrape_page, url) for url in urls]

            # Process results as they complete
            for future in as_completed(futures):
                try:
                    # Results are processed in the scrape_page method
                    future.result()
                except Exception as e:
                    logging.error(f"Error in parallel scraping: {e}")

        # Process data after all scraping is complete
        self.post_process_data()

    def scrape_deep_section(self, base_url, max_depth=1):
        """
        Efficiently scrape a section using breadth-first crawling with parallel processing.
        Uses a queue-based approach instead of recursion for better performance.
        """
        # Initialize a driver for this section
        driver = webdriver.Chrome(options=self.options)

        try:
            # Use a queue for breadth-first traversal
            url_queue = queue.Queue()
            url_queue.put((base_url, 0))  # (url, depth)

            # Keep track of URLs we've queued to avoid duplicates
            queued_urls = {base_url}

            # Process URLs in breadth-first order
            while not url_queue.empty():
                current_url, depth = url_queue.get()

                # Skip if we've reached max depth
                if depth > max_depth:
                    continue

                # Mark as visited
                self.visited_urls.add(current_url)

                # Scrape the page
                page_data, links = self.scrape_page_with_driver(driver, current_url)

                # Thread-safe addition of data
                with self.data_lock:
                    self.data.extend(page_data)

                # Add child links to queue if within depth limit
                if depth < max_depth:
                    # Filter links to only include relevant ones
                    relevant_links = []
                    for link in links:
                        # Apply additional filtering for deeper crawls
                        if depth > 0:
                            # Only include pages that are likely product pages
                            keywords = ['card', 'insure', 'invest', 'save', 'loan', 'borrow', 'wealth']
                            if not any(keyword in link.lower() for keyword in keywords):
                                continue

                        # Add if not already queued
                        if link not in queued_urls:
                            relevant_links.append(link)
                            queued_urls.add(link)

                    # Add filtered links to queue
                    for link in relevant_links:
                        url_queue.put((link, depth + 1))

                    # Don't overload the server - small delay between batches
                    if len(relevant_links) > 0:
                        time.sleep(0.2)

        except Exception as e:
            logging.error(f"Error in deep scraping {base_url}: {e}")
        finally:
            driver.quit()

    def scrape_deep_sections_parallel(self, base_urls, max_depth=2):
        """Scrape multiple sections in parallel."""
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # Create a future for each base URL
            futures = [
                executor.submit(self.scrape_deep_section, url, max_depth)
                for url in base_urls
            ]

            # Wait for all futures to complete
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    logging.error(f"Error in parallel deep scraping: {e}")

        # Process all collected data
        self.post_process_data()

    def post_process_data(self):
        """Process and save the collected data."""
        if not self.data:
            logging.warning("No data collected to process.")
            return

        df = pd.DataFrame(self.data)

        # Apply cleaning to all text columns to ensure consistency
        for col in ['title', 'subtitle', 'subheader', 'text']:
            df[col] = df[col].apply(lambda x: self.clean_text(str(x)) if pd.notna(x) else '')

        # Filter out invalid entries and duplicates
        df = df[df['text'].apply(self.is_valid_text)]
        df = df.drop_duplicates(subset=['text'])

        # Additional filtering for unwanted content patterns
        def contains_unwanted_patterns(text):
            unwanted_patterns = [
                r'back to main',
                r'copyright',
                r'all rights reserved',
                r'privacy notice',
                r'cookie policy',
                r'united overseas bank'
            ]

            for pattern in unwanted_patterns:
                if re.search(pattern, text, re.IGNORECASE):
                    return True
            return False

        df = df[~df['text'].apply(contains_unwanted_patterns)]

        # Sort by URL and subheader
        df = df.sort_values(['url', 'subheader'])

        # Save to CSV with explicit UTF-8 encoding
        output_file = f"{self.output_dir}/uob_data_cleaned.csv"

        # Using pandas to write with appropriate encoding
        df.to_csv(output_file, index=False, encoding='utf-8-sig', quoting=csv.QUOTE_ALL)

        # Also save to Excel as an alternative format that might handle encoding better
        excel_file = f"{self.output_dir}/uob_data_cleaned.xlsx"
        df.to_excel(excel_file, index=False, engine='openpyxl')

        print(f"Data successfully saved to {output_file} and {excel_file}")
        print(f"Total entries: {len(df)}")
        print(f"Total unique URLs scraped: {len(df['url'].unique())}")

    def close(self):
        """Clean up resources."""
        pass  # Drivers are now managed per-thread

if __name__ == "__main__":
    # 1. Create the output directory first:
    output_dir = "uob_data"
    os.makedirs(output_dir, exist_ok=True)

    # 2. Then configure logging, using the existing output directory:
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(os.path.join(output_dir, f"scraper_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")),
            logging.StreamHandler()
        ]
    )

    # Manually placed URLs for the main categories
    main_urls = [
        "https://www.uob.com.sg/personal/index.page",                 # /personal/index.page
        "https://www.uob.com.sg/personal/finlit/index.page",            # /personal/finlit/index.page
        "https://www.uob.com.sg/personal/digital-banking/pib/security/what-to-do-if-you-have-been-scammed/index.page",  # /personal/digital-banking/pib/security/what-to-do-if-you-have-been-scammed/index.page
        "https://www.uob.com.sg/personal/save/protect-your-money.page",   # /personal/save/protect-your-money.page
        "https://www.uob.com.sg/personal/digital-banking/pib/changes-to-sms-alerts.page", # /personal/digital-banking/pib/changes-to-sms-alerts.page
        "https://www.uob.com.sg/personal/customer-service/index.page",  # /personal/customer-service/index.page
        "https://www.uob.com.sg/personal/save/index.page",             # /personal/save/index.page
        "https://www.uob.com.sg/personal/save/index.page?filter=everyday-use",
        "https://www.uob.com.sg/personal/save/index.page?filter=consistent-saving",
        "https://www.uob.com.sg/personal/save/index.page?filter=kids",
        "https://www.uob.com.sg/personal/save/index.page?filter=foreign-currency",
        "https://www.uob.com.sg/personal/save/index.page?filter=fixed-deposits",
        "https://www.uob.com.sg/personal/save/everyday-accounts/one-account.page",
        "https://www.uob.com.sg/personal/save/savings-accounts/stash-account.page",
        "https://www.uob.com.sg/personal/save/foreign-currency-accounts/fxplus.page",
        "https://www.uob.com.sg/personal/save/savings-accounts/ladys-savings-account.page",
        "https://www.uob.com.sg/personal/save/everyday-accounts/krisflyer-uob-account.page",
        "https://www.uob.com.sg/personal/promotions/save/index.page?s_cid=grm:sg:owned:onl:pweb:oao:bu:na:casa:010425-300625:Q2SAVE25:na:na&vid=na&pid=CD07Q2SAVE25",
        "https://www.uob.com.sg/personal/save/one-account-tax-saver.page",
        "https://www.uob.com.sg/personal/save/deputy-donee-account.page",
        "https://www.uob.com.sg/personal/save/services/egiro.page",
        "https://www.uob.com.sg/personal/cards/services/merchant-services.page",
        "https://www.uob.com.sg/personal/cards/news-and-announcements.page",
        "https://www.uob.com.sg/personal/apply-now.page?filter=cards",
        "https://www.uob.com.sg/personal/borrow/index.page",
        "https://www.uob.com.sg/personal/borrow/personal-financing/index.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/index.page",
        "https://www.uob.com.sg/personal/borrow/car-financing/index.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/index.page#complete-your-loan-application",
        "https://www.uob.com.sg/personal/borrow/car-financing/index.page#loan-application",
        "https://www.uob.com.sg/personal/borrow/personal-financing/cashplus.page",
        "https://www.uob.com.sg/personal/borrow/personal-financing/personal-loan.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/private-home-loan.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/hdb-home-loan.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/international-property-loan.page",
        "https://www.uob.com.sg/personal/borrow/personal-financing/balance-transfer.page",
        "https://www.uob.com.sg/personal/borrow/personal-financing/debt-consolidation-plan.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/managing-your-uob-property-loan.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/referral-programme.page",
        "https://www.uob.com.sg/personal/borrow/property-loans/private-home-loan.page#benefits",
        "https://www.uob.com.sg/personal/highlights/solutions/home-and-property/index.page",
        "https://www.uob.com.sg/personal/customer-service/loans.page#how-do-i-apply-for-a-car-loan",
        "https://www.uob.com.sg/personal/borrow/bankers-guarantee.page",
        "https://www.uob.com.sg/personal/apply-now.page?filter=loans",
        "https://www.uob.com.sg/personal/invest/singapore-government-securities.page",
        "https://www.uob.com.sg/personal/invest/unit-trusts/united-cio-funds.page",
        "https://www.uob.com.sg/personal/digital-banking/wealth-on-tmrw.page?s_cid=grm:sg:owned:pweb:investmegamenutile%20:na:bu:na:simpleinv:150823-evergreen:simpleinvest:na:na&vid=na",
        "https://www.uob.com.sg/personal/digital-banking/index.page#whats-new",
        "https://www.uob.com.sg/personal/digital-banking/index.page#other-features",
        "https://www.uob.com.sg/personal/digital-banking/index.page#tmrw-download-now",
        "https://www.uob.com.sg/personal/digital-banking/overview.page",
        "https://www.uob.com.sg/personal/digital-banking/pib/index.page#services",
        "https://www.uob.com.sg/personal/digital-banking/pib/security/how-uob-protects-you/index.page",
        "https://www.uob.com.sg/personal/digital-banking/phone-banking.page",
        "https://www.uob.com.sg/personal/digital-banking/nets-contactless.page",
        "https://www.uob.com.sg/personal/digital-banking/pib/forgot-digital-username-or-password.page",
        "https://www.uob.com.sg/personal/customer-service/tmrw-user-guide/index.page",
        "https://www.uob.com.sg/personal/customer-service/pib-tmrw/index.page",
        "https://www.uob.com.sg/wealthbanking/index.page",
        "https://www.uob.com.sg/privilegebanking/index.page",
        "https://www.uob.com.sg/privilegereserve/index.page",
        "https://www.uob.com.sg/private/index.page",
        "https://www.uob.com.sg/business/index.page",
        "https://www.uobgroup.com/foreign-direct-investment/index.page",
        "https://www.uobgroup.com/asean-insights/index.page",
        "https://www.uobgroup.com/industry-insights/index.page",
        "https://www.uobgroup.com/sustainable-solutions/index.page",
        "https://www.uobgroup.com/uobgroup/index.page",
        "https://www.uob.com.sg/online-branch/index.page",
        "https://www.uobgroup.com/techecosystem",
        "https://www.uobgroup.com/uobworld/index.page",
        "https://www.uobam.com.sg/",
        "https://www.uoi.com.sg/",
        "https://www.uobgroup.com/sustainability/index.page",
        "https://www.uob.com.sg/personal/digital-banking/pib/security/index.page",
        "https://www.uob.com.sg/personal/digital-banking/pib/security/how-uob-protects-you/index.page",
        "https://www.uob.com.sg/personal/digital-banking/pib/security/index.page#alers-announcements",
        "https://www.uobgroup.com/uobgroup/contact-us/index.page",
        "https://www.uob.com.sg/online-branch/locate-us.page",
        "https://www.uob.com.sg/personal/customer-service/application-forms.page",
        "https://www.uob.com.sg/personal/chat.page",
        "https://pib.uob.com.sg/PIBLogin/Public/processPreCapture.do?keyId=lpc",
        "https://secure.uobam.com.sg/login?lid=uobam-invest-login",
        "https://ocoe.uob.com.sg/login",
        "https://pib.uob.com.sg/Rewards/",
        "https://www.uob.com.sg/personal/highlights/life-moments/index.page",
        "https://www.uob.com.sg/personal/highlights/life-moments/adulting/index.page",
        "https://www.uob.com.sg/personal/highlights/life-moments/exploring-the-world.page",
        "https://www.uob.com.sg/personal/highlights/life-moments/growing-your-money.page",
        "https://www.uob.com.sg/personal/highlights/life-moments/getting-married.page",
        "https://www.uob.com.sg/personal/highlights/life-moments/starting-a-family.page",
        "https://www.uob.com.sg/personal/highlights/life-moments/planning-your-retirement.page",
        "https://www.uob.com.sg/personal/highlights/solutions/for-the-ladies.page",
        "https://www.uob.com.sg/personal/highlights/solutions/sustainable-living.page",
        "https://www.uob.com.sg/personal/solutions/wellness/index.page",
        "https://www.uob.com.sg/personal/highlights/solutions/home-and-property/index.page",
        "https://www.uob.com.sg/personal/highlights/solutions/drivers-and-commuters.page",
        "https://www.uob.com.sg/personal/highlights/solutions/index.page",
        "https://www.uob.com.sg/uobgroup/url-redirection.page?reURL=https://thediningadvisor.com/home",
        "https://www.uob.com.sg/uobgroup/url-redirection.page?reURL=https://thetravelinsider.co/sg/en",
        "https://www.uob.com.sg/shopuob",
        "https://www.uob.com.sg/personal/digital-banking/index.page",
        "https://www.uob.com.sg/personal/cards/index.page",
        "https://www.uob.com.sg/personal/cards/index.page?filter=cashback",
        "https://www.uob.com.sg/personal/cards/index.page?filter=travel",
        "https://www.uob.com.sg/personal/cards/index.page?filter=rewards",
        "https://www.uob.com.sg/personal/cards/index.page?filter=privilege",
        "https://www.uob.com.sg/personal/cards/index.page#debit-cards",
        "https://www.uob.com.sg/personal/cards/supplementary-card.page",
        "https://www.uob.com.sg/personal/cards/rewards/index.page",
        "https://www.uob.com.sg/personal/cards/card-privileges/fuel-power-index.page",
        "https://www.uob.com.sg/personal/cards/card-privileges/uob-dollar.page",
        "https://uob.com.sg/transit",
        "https://www.uob.com.sg/personal/cards/services/payment-facility.page",
        "https://www.uob.com.sg/personal/cards/services/smart-pay.page",
        "https://www.uob.com.sg/personal/cards/services/recurring-bill-payment.page",
        "https://www.uob.com.sg/personal/cards/services/credit-limit-review.page",
        "https://www.uob.com.sg/personal/cards/services/card-alerts.page",
        "https://www.uob.com.sg/personal/customer-service/credit-card.page",
        "https://www.uob.com.sg/personal/cards/services/index.page",
        "https://www.uob.com.sg/personal/promotions/cards/sign-up-offers/index.page",
        "https://www.uob.com.sg/personal/cards/services/card-activation.page",
        "https://www.uob.com.sg/personal/cards/services/overseas-card-use.page",
        "https://forms.uob.com/sg/apply/status",
        "https://forms.uob.com.sg/property/apply",
        "https://forms.uob.com.sg/property/calculator/",
        "https://forms.uob.com.sg/property/valuation/",
        "https://forms.uob.com.sg/personal/services/carloans/contact-us.html",
        "https://www.uob.com.sg/personal/invest/index.page",
        "https://www.uob.com.sg/personal/invest/wealth-insights/index.page#daily-updates",
        "https://www.uob.com.sg/personal/invest/wealth-insights/index.page",
        "https://www.uob.com.sg/personal/invest/market-outlook/index.page?i_cid=grm:sg:owned:onl:pweb:na:tx:na:uobins:020724-evergreen:moy:pbinvestdropdown:na&vid=na",
        "https://www.uob.com.sg/personal/invest/wealth-approach/index.page",
        "https://www.uob.com.sg/personal/invest/unit-trusts/index.page",
        "https://www.uob.com.sg/personal/invest/gold-and-silver.page",
        "https://www.uob.com.sg/personal/invest/cpf-investment-account.page",
        "https://www.uob.com.sg/personal/invest/srs-account.page",
        "https://www.uob.com.sg/personal/invest/sustainable-investing/index.page",
        "https://www.uob.com.sg/personal/invest/income-builder.page",
        "https://www.uob.com.sg/personal/invest/unit-trusts/fund-selector.page",
        "https://www.uob.com.sg/personal/invest/product-providers.page",
        "https://www.uob.com.sg/personal/invest/cpfis-srs-corporate-actions-information.page",
        "https://www.uob.com.sg/web-resources/personal/pdf/personal/best-execution-customer-disclosure.pdf",
        "https://forms.uob.com.sg/uobforms/POR/index.page",
        "https://www.uob.com.sg/personal/insure/general/index.page?filter=accidents-and-health",
        "https://www.uob.com.sg/personal/insure/general/index.page?filter=travel",
        "https://www.uob.com.sg/personal/insure/general/insuredrive.page",
        "https://www.uob.com.sg/personal/insure/general/index.page?filter=home-and-helper",
        "https://www.uob.com.sg/personal/insure/general/index.page",
        "https://www.uob.com.sg/personal/insure/life/retirement/index.page",
        "https://www.uob.com.sg/personal/insure/life/savings.page",
        "https://www.uob.com.sg/personal/insure/life/protection/index.page",
        "https://www.uob.com.sg/personal/insure/life/legacy.page",
        "https://www.uob.com.sg/personal/insure/life/index.page",
        "https://www.uob.com.sg/personal/insure/life/protection/creditsure-plus.page",
        "https://www.uob.com.sg/personal/insure/life/pokemon.page",
        "https://www.uob.com.sg/personal/digital-banking/index.page",
        "https://www.uob.com.sg/personal/digital-banking/paynow.page"
    ]


    # Create a scraper with 5 parallel workers
    scraper = UOBScraper(headless=True, max_workers=5)

    try:
        # First, scrape the main category URLs in parallel
        logging.info("Scraping main category pages...")
        scraper.scrape_urls(main_urls)

        # Then scrape product sections in parallel with deeper crawling
        logging.info("Starting deep section scraping...")
        scraper.scrape_deep_sections_parallel(deep_urls, max_depth=2)

        # Final processing is done in the scrape methods
        logging.info("Scraping completed successfully.")
    except Exception as e:
        logging.error(f"Error during scraping: {e}")
    finally:
        scraper.close()

Data successfully saved to uob_data/uob_data_cleaned.csv and uob_data/uob_data_cleaned.xlsx
Total entries: 4511
Total unique URLs scraped: 141
Data successfully saved to uob_data/uob_data_cleaned.csv and uob_data/uob_data_cleaned.xlsx
Total entries: 4511
Total unique URLs scraped: 141
